In [1]:
# https://signals.boulderinvestment.tech/#map
# https://pypi.org/project/lppls/

from lppls import lppls
import numpy as np
import pandas as pd
from datetime import datetime as dt
from matplotlib import pyplot as plt


(1338, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2020-09-22,1600732800000,0.8500,7.0000,0.8500,5.3193,31441883.16
2020-09-23,1600819200000,5.3279,5.3600,3.4000,3.5350,12620608.01
2020-09-24,1600905600000,3.5305,4.8873,3.4033,4.6411,7107843.86
2020-09-25,1600992000000,4.6366,5.0234,4.2001,4.7134,3886096.71


In [59]:
PAIR = 'ALGO_USDT'
TIMEFRAME = '8h'

EXCHANGE = 'binance'

odf = pd.read_json(f'/media/mu6mula/Data/Crypto-Data-Feed/freq-user-data/data/{EXCHANGE}/{PAIR}-{TIMEFRAME}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
# ).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
# ).drop('timestamp'
)#.set_index('dtime').sort_index()

odf.head(5)

,timestamp,open,high,low,close,volume
0,1577836800000,0.2164,0.2202,0.2152,0.2158,792393.81
1,1577865600000,0.2156,0.2241,0.2149,0.2241,976021.23
2,1577894400000,0.2241,0.2248,0.2195,0.2201,553113.69
3,1577923200000,0.2201,0.2210,0.2154,0.2155,630494.59
4,1577952000000,0.2155,0.2195,0.2155,0.2172,447858.60


In [60]:

"""
  Fit Model
  Fit your data to the LPPL model
"""



# time = [pd.Timestamp.toordinal(dt.strptime(t1, '%Y-%m-%d')) for t1 in data['date']]
# time = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)

dtime = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
time = [pd.Timestamp.toordinal(d) for d in dtime]
price = np.log(odf['close'].values)
observations = np.array([time, price])
MAX_SEARCHES = 25
lppls_model = lppls.LPPLS(observations=observations)
tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(MAX_SEARCHES)

In [61]:

"""
  Compute Confidence Indicator
  Run computations for lppl and compute the confidence indicator
"""

res = lppls_model.mp_compute_nested_fits(
  workers=8,
  window_size=120, 
  smallest_window_size=30, 
  outer_increment=1, 
  inner_increment=5, 
  max_searches=25
)

"""
  Save Confidence Indicator
  Run computations for lppl
"""

res_df = lppls_model.compute_indicators(res)



  0%|          | 0/4692 [00:00<?, ?it/s]/home/mu6mula/miniconda3/envs/py310/lib/python3.10/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/mu6mula/miniconda3/envs/py310/lib/python3.10/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/mu6mula/miniconda3/envs/py310/lib/python3.10/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/mu6mula/miniconda3/envs/py310/lib/python3.10/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/mu6mula/miniconda3/envs/py310/lib/python3.10/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1

In [65]:
res_df

,time,price,pos_conf,neg_conf,_fits
0,737464.0,-1.075580,0.0,0.0,"[{'tc': 737464.3542680293, 'm': 1.054321539996..."
1,737465.0,-1.114132,0.0,0.0,"[{'tc': 737492.9928562404, 'm': -2.34215430689..."
2,737465.0,-1.096614,0.0,0.0,"[{'tc': 737493.1268039483, 'm': -2.34251348129..."
3,737465.0,-1.025827,0.0,0.0,"[{'tc': 737493.0520199486, 'm': -2.34621687183..."
4,737466.0,-1.070608,0.0,0.0,"[{'tc': 737492.1449922221, 'm': -2.39178947285..."
...,...,...,...,...,...
4687,739027.0,-1.664422,0.0,0.0,"[{'tc': 739019.5426036427, 'm': 1.651025803265..."
4688,739027.0,-1.669719,0.0,0.0,"[{'tc': 739023.4738881327, 'm': 1.725883339958..."
4689,739027.0,-1.669188,0.0,0.0,"[{'tc': 738972.3224897105, 'm': -2.61910394460..."
4690,739028.0,-1.675578,0.0,0.0,"[{'tc': 739021.7696593665, 'm': 1.087415326025..."


In [63]:

"""
  Save Fit
  Save and show your fitted results
"""

time_ord = [pd.Timestamp.fromordinal(d) for d in lppls_model.observations[0, :].astype('int32')]
# time_ord = [pd.Timestamp(d) for d in lppls_model.observations[0, :]]

t_obs = lppls_model.observations[0, :]
lppls_fit = [lppls_model.lppls(t, tc, m, w, a, b, c1, c2) for t in t_obs]
price = lppls_model.observations[1, :]

first = t_obs[0]
last = t_obs[-1]

ord = res_df['time'].astype('int32')
# ord = res_df['time']
ts = [pd.Timestamp.fromordinal(d) for d in ord]
# ts = [pd.Timestamp(d) for d in ord]


In [84]:
res_df.columns

Index(['time', 'price', 'pos_conf', 'neg_conf', '_fits'], dtype='object')

In [86]:
res_df[['time', 'price', 'pos_conf', 'neg_conf']].to_csv(f"./res_df/{PAIR}-{TIMEFRAME}.csv")

In [81]:


fig, (ax1) = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(14, 8))

ax1.plot(time_ord, price, label='price', color='black', linewidth=0.75)
ax1.plot(time_ord, lppls_fit, label='lppls fit', color='blue', alpha=0.75)
ax1.grid(which='major', axis='both', linestyle='--')
ax1.set_ylabel('ln(p)')
ax1.legend(loc=2)

plt.xticks(rotation=45)
plt.savefig(f"./images/fitting/{PAIR}-{TIMEFRAME}.png")
plt.clf()

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(18, 10))

# plot pos bubbles
ax1_0 = ax1.twinx()
ax1.plot(ts, res_df['price'], color='black', linewidth=0.5)
ax1_0.plot(ts, res_df['pos_conf'], label='bubble indicator (pos)', color='red', alpha=0.75)
pfres_df = res_df[res_df['pos_conf'] > 0]
for i in pd.Series(ts)[res_df['pos_conf'] > 0]:
    ax1_0.axvline(i, lw=0.75, color='orange', alpha=0.25)
# plot neg bubbles
ax2_0 = ax2.twinx()
ax2.plot(ts, res_df['price'], color='black', linewidth=0.5)
ax2_0.plot(ts, res_df['neg_conf'], label='bubble indicator (neg)', color='green', alpha=0.75)

for i in pd.Series(ts)[res_df['neg_conf'] > 0]:
    ax2_0.axvline(i, lw=0.75, color='teal', alpha=0.25)
    
# set grids
ax1.grid(which='major', axis='both', linestyle='--')
ax2.grid(which='major', axis='both', linestyle='--')

# set labels
ax1.set_ylabel('ln(p)')
ax2.set_ylabel('ln(p)')

ax1_0.set_ylabel('bubble indicator (pos)')
ax2_0.set_ylabel('bubble indicator (neg)')

ax1_0.legend(loc=2)
ax2_0.legend(loc=2)

plt.xticks(rotation=45)
plt.savefig(f"./images/confidence/{PAIR}-{TIMEFRAME}.png")
plt.clf()

<Figure size 1400x800 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>